In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
from PIL import Image
import json
from glob import glob
import numpy as np
import tensorflow as tf

In [ ]:
with open('dev.json', 'r') as fp:
    devsamples = json.load(fp)
json_sample = devsamples
print(len(json_sample['images']))
print(json_sample.keys())
for i in range(2):
    print('#####')
    print(json_sample['images'][i])

In [ ]:
images = {}
for image in devsamples['images']:
    images[image['id']] = {'file_name': image['file_name'], 
                           'width': image['width'], 
                           'height': image['height'], 
                           'annotations': []}
for ann in devsamples['annotations']:
    images[ann['image_id']]['annotations'].append(ann)

In [ ]:
print(len(images.keys()))
h, w = [], []
for img_id in list(images):
    w.append(images[img_id]['width'])
    h.append(images[img_id]['height'])

print('max w', np.max(w), 'max h', np.max(h))
plt.hist(w, 20)
plt.show()
plt.hist(h, 20)
plt.show()


In [ ]:

print(len(images.keys()))
for img_id in list(images)[:2]:
    print('CCCCCCCCCCCCCC')
    print(images[img_id])
    seg_mask = np.zeros((images[img_id]['width'],images[img_id]['height']), dtype=int)
    
    orig_img = np.array(Image.open(images[img_id]['file_name']).convert('RGB'))
    orig_img = orig_img.T
    
    for ann in images[img_id]['annotations']:
        #print(type(ann['bbox']), ann['bbox'][0])
        current_bbox = np.asarray(ann['bbox'], dtype = np.int32)
        x1, x2 = current_bbox[0], current_bbox[0] + current_bbox[2]
        y1, y2 = current_bbox[1], current_bbox[1] + current_bbox[3]
        seg_mask[x1:x2, y1:y2] = ann['category_id']*40
        
        seg_mask[x1, y1:y2] = 255
        seg_mask[x2, y1:y2] = 255
        seg_mask[x1:x2, y1] = 255
        seg_mask[x1:x2, y2] = 255
        
        orig_img[:, x1-1, y1-1:y2+1] = 100
        orig_img[:, x2+1, y1-1:y2+1] = 100
        orig_img[:, x1-1:x2+1, y1-1] = 100
        orig_img[:, x1-1:x2+1, y2+1] = 100
        
    orig_img = Image.fromarray(orig_img.T.astype(dtype=np.uint8),mode='RGB')
    with tf.gfile.Open(images[img_id]['file_name'][:-4]+'mod1.jpg', mode='w') as f:
        orig_img.save(f, 'JPEG')    

    seg_mask = seg_mask.T    
    seg_img = Image.fromarray(seg_mask.astype(dtype=np.uint8))
    segfilename = 'SegmentationClass/'+images[img_id]['file_name'][:-4]+'.png'
    with tf.gfile.Open(segfilename, mode='w') as f:
        seg_img.save(f, 'PNG')

In [ ]:
import glob
path = 'test/'

files = [f for f in glob.glob(path + "**/*.pdf", recursive=True)]
print(type(files))
print(len(files))
for f in files:
    print(f)
file_names = [f.split('/')[-1].split('.')[-2] for f in files]
print(file_names)
print()